In [1]:
from model import MLP

model = MLP()

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

LinW layers:

Depth 0: LinW(in_features=512, out_features=512, bias=True)
Depth 1: LinW(in_features=512, out_features=512, bias=True)


In [2]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader.dataset)
    accuracy = 100. * correct / total
    return train_loss, accuracy

In [3]:
import torch

def evaluate(device, model, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


/home/andrea/anaconda3/envs/dlea/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402412426/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10, Training Loss: 1.8271, Training Accuracy: 33.33%, Test accuracy: 36.13%
Epoch 2/10, Training Loss: 1.6442, Training Accuracy: 40.63%, Test accuracy: 39.18%
Epoch 3/10, Training Loss: 1.5629, Training Accuracy: 43.49%, Test accuracy: 45.76%
Epoch 4/10, Training Loss: 1.4941, Training Accuracy: 46.47%, Test accuracy: 46.67%
Epoch 5/10, Training Loss: 1.4457, Training Accuracy: 48.18%, Test accuracy: 48.36%
Epoch 6/10, Training Loss: 1.4058, Training Accuracy: 49.57%, Test accuracy: 49.13%
Epoch 7/10, Training Loss: 1.3506, Training Accuracy: 51.61%, Test accuracy: 49.03%
Epoch 8/10, Training Loss: 1.3119, Training Accuracy: 52.92%, Test accuracy: 50.55%
Epoch 9/10, Training Loss: 1.2652, Training Accuracy: 54.44%, Test accuracy: 49.43%
Epoch 10/10, Training Loss: 1.2278, Training Accuracy: 55.86%, Test accuracy: 49.46%
